In [3]:
import pandas as pd
import re

def extract_russian_name(text):
    """Извлекает русское имя с учетом всех исключений"""
    if not isinstance(text, str) or not text.strip():
        return None
    
    text = re.sub(r'^\s*Re:\s*', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\(.*?\)', '', text)
    
    capital_words = re.findall(r'[А-ЯЁ][а-яё]+', text)
    if len(capital_words) > 1:
        match = re.search(
            r'(?:[А-ЯЁ][а-яё]+\s+){1,2}([А-ЯЁ][а-яё]+\s+[А-ЯЁ][а-яё]+)', 
            text
        )
        if match:
            return ' '.join(match.group(0).strip().split())
    return None

def clean_id_hash(text):
    """Удаляет статусные слова из id_hash"""
    if not isinstance(text, str):
        return text
        
    text = re.sub(
        r'\b(пропал[а-яё]*|жив[а-яё]*|погиб[а-яё]*|пропаж[а-яё]*)\b', 
        '', 
        text, 
        flags=re.IGNORECASE
    )
    return text.strip()

def process_file():
    try:
        # Чтение файла с определением кодировки
        try:
            df = pd.read_csv('updated_file.csv', encoding='utf-8')
        except UnicodeDecodeError:
            df = pd.read_csv('updated_file.csv', encoding='cp1251')

        # Проверка обязательных колонок
        required_columns = [
            'id_hash', 'gender', 'age', 'location', 'status',
            'date_search', 'date_of_loss', 'last_search_date',
            'search_period', 'head', 'date_of_find'
        ]
        
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            print(f"Ошибка: отсутствуют колонки {missing_columns}")
            return

        # Сохраняем оригинальные данные для статистики
        initial_count = len(df)
        
        # Обработка имен
        df['original_id_hash'] = df['id_hash']
        df['extracted_name'] = df['head'].apply(extract_russian_name)
        df.loc[df['extracted_name'].notna(), 'id_hash'] = df['extracted_name']
        
        # Очистка от статусных слов
        df['id_hash'] = df['id_hash'].apply(clean_id_hash)

        # Поля для определения дубликатов
        duplicate_columns = [
            'id_hash', 'gender', 'age', 'location', 'status',
            'date_search', 'date_of_loss', 'last_search_date',
            'search_period', 'head', 'date_of_find'
        ]
        
        # Удаление дубликатов
        duplicates_mask = df.duplicated(subset=duplicate_columns, keep='first')
        removed_count = duplicates_mask.sum()
        df = df[~duplicates_mask]

        # Удаление служебных колонок
        df.drop(
            columns=['original_id_hash', 'extracted_name'],
            inplace=True,
            errors='ignore'
        )

        # Сохранение результата
        df.to_csv('updated_file.csv', index=False, encoding='utf-8')
        
        print("Файл успешно обработан. Статистика:")
        print(f"Всего строк до обработки: {initial_count}")
        print(f"Удалено дубликатов: {removed_count}")
        print(f"Оставлено уникальных записей: {len(df)}")
        print("\nПример обработанных данных:")
        print(df.head())

    except Exception as e:
        print(f"Ошибка обработки: {str(e)}")

if __name__ == "__main__":
    process_file()
df=pd.read_csv('updated_file.csv')
print(df['id_hash'].value_counts(dropna=False))

Файл успешно обработан. Статистика:
Всего строк до обработки: 40016
Удалено дубликатов: 0
Оставлено уникальных записей: 40016

Пример обработанных данных:
   Unnamed: 0                                            id_hash gender   age  \
0           0  002a7a0fd2047c4812ee06974cfec87266c73307ac4d29...    жен  65.0   
1           1  008c48640c96927ecb37bf877179e2e506052ea235e4d7...    муж  38.0   
2           2  00fdf06a413b627f8a53c77ce9dcde274b94aa895d6f15...    муж   NaN   
3           3  012742c048aa46328fbca7f18376b6db990499a31ca158...    муж  69.0   
4           4  0228b07e42be793dc3259b75fe30468085922a0959c24f...    жен  45.0   

          location     status         date_search  date_of_loss  \
0              NaN  пропал(а)  19 сен 2023, 00:42           NaN   
1  пгт. Кавалерово  пропал(а)  12 июн 2024, 06:25   9 июня 2024   
2              NaN        NaN  15 авг 2018, 02:00           NaN   
3              NaN  пропал(а)  15 сен 2023, 06:57  24 июня 2023   
4              NaN     

In [38]:
import pandas as pd
import re
from tqdm import tqdm

# Списки для фильтрации географических названий
GEO_WORDS = {
    'область', 'район', 'город', 'деревня', 'село', 'посёлок', 
    'республика', 'край', 'округ', 'улица', 'проспект', 'аллея',
    'переулок', 'площадь', 'шоссе', 'набережная', 'бульвар', 'микрорайон'
}

REGIONS = {
    'москва', 'санкт-петербург', 'новосибирск', 'екатеринбург', 'казань',
    'нижний новгород', 'челябинск', 'самара', 'омск', 'ростов-на-дону',
    'красноярск', 'пермь', 'воронеж', 'волгоград', 'краснодар'
}

def is_geo_name(text):
    """Проверяет, является ли текст географическим названием"""
    if not isinstance(text, str):
        return False
    
    text_lower = text.lower()
    words = text_lower.split()
    
    # Проверка по ключевым словам
    if any(geo_word in text_lower for geo_word in GEO_WORDS):
        return True
    
    # Проверка на регионы (только для одиночных слов)
    if len(words) == 1 and words[0] in REGIONS:
        return True
        
    return False

def extract_name_from_text(text):
    """Извлекает имя из текста с фильтрацией гео-названий"""
    if not isinstance(text, str) or not text.strip():
        return None
    
    # Очистка текста
    text = re.sub(r'^\s*Re:\s*', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\(.*?\)', '', text)
    text = re.sub(r'[«»"“”]', '', text)
    text = re.sub(r'\d+', '', text)  # Удаление чисел
    
    # Удаление статусных слов
    text = re.sub(
        r'\b(пропал[а-яё]*|жив[а-яё]*|погиб[а-яё]*|пропаж[а-яё]*|стоп\b)', 
        '', 
        text, 
        flags=re.IGNORECASE
    )
    
    # Паттерны для поиска имен
    patterns = [
        r'([А-ЯЁ][а-яё]+(?:\s+[А-ЯЁ][а-яё]+){1,2})',  # ФИО или Фамилия+Имя
        r'([А-ЯЁ][а-яё]+\s+[А-ЯЁ]\.\s*[А-ЯЁ]\.)',    # Фамилия с инициалами
    ]
    
    for pattern in patterns:
        matches = list(re.finditer(pattern, text))
        if matches:
            # Берем последнее совпадение (чаще всего в конце текста)
            name = matches[-1].group(0).strip()
            name = re.sub(r'([А-ЯЁ])\.\s*([А-ЯЁ])\.', r'\1. \2.', name)
            if not is_geo_name(name):
                return name
    return None

def process_file(input_file, output_file):
    try:
        # Загрузка данных
        try:
            df = pd.read_csv(input_file, encoding='utf-8')
        except UnicodeDecodeError:
            df = pd.read_csv(input_file, encoding='cp1251')
        
        # Проверка колонок
        required_cols = ['head', 'content', 'id_hash']
        missing_cols = [col for col in required_cols if col not in df.columns]
        if missing_cols:
            print(f"Ошибка: отсутствуют колонки {missing_cols}")
            return

        # 1. Приоритетный анализ поля head
        tqdm.pandas(desc="Анализ head")
        df['name_from_head'] = df['head'].progress_apply(extract_name_from_text)
        
        # 2. Анализ content ТОЛЬКО для строк без имени в head
        no_name_mask = df['name_from_head'].isna()
        if no_name_mask.any():
            tqdm.pandas(desc="Анализ content")
            df.loc[no_name_mask, 'name_from_content'] = df.loc[no_name_mask, 'content'].progress_apply(extract_name_from_text)
        
        # 3. Объединение результатов с абсолютным приоритетом head
        df['extracted_name'] = df['name_from_head']  # Всегда берем из head если есть
        
        # 4. Замена id_hash с фильтрацией гео-названий
        valid_mask = df['extracted_name'].notna() & ~df['extracted_name'].apply(is_geo_name)
        df.loc[valid_mask, 'id_hash'] = df.loc[valid_mask, 'extracted_name']
        
        # Статистика
        stats = {
            'total': len(df),
            'from_head': df['name_from_head'].notna().sum(),
            'from_content': df['name_from_content'].notna().sum(),
            'geo_filtered': df['extracted_name'].apply(is_geo_name).sum(),
            'replaced': valid_mask.sum()
        }
        
        print(f"\nСтатистика обработки:")
        print(f"Всего строк: {stats['total']}")
        print(f"Найдено в head: {stats['from_head']} ({stats['from_head']/stats['total']:.1%})")
        print(f"Найдено в content: {stats['from_content']} (дополнительно)")
        print(f"Отфильтровано гео-названий: {stats['geo_filtered']}")
        print(f"Итоговых замен id_hash: {stats['replaced']} ({stats['replaced']/stats['total']:.1%})")
        
        # Примеры изменений
        if valid_mask.any():
            print("\nПримеры успешных замен:")
            print(df.loc[valid_mask, ['head', 'content', 'id_hash']].head(3).to_string(index=False))
        
        # Сохранение
        cols_to_drop = ['name_from_head', 'name_from_content', 'extracted_name']
        df.drop(columns=[col for col in cols_to_drop if col in df.columns], inplace=True)
        df.to_csv(output_file, index=False, encoding='utf-8')
        
        print(f"\nРезультат сохранен в: {output_file}")
        
    except Exception as e:
        print(f"Ошибка обработки: {str(e)}")

if __name__ == "__main__":
    process_file("updated_file.csv", "updated_file.csv")
    df=pd.read_csv('updated_file.csv')
    print(df['id_hash'].value_counts(dropna=False))

Анализ content: 100%|██████████| 11140/11140 [00:00<00:00, 88436.51it/s]



Статистика обработки:
Всего строк: 40016
Найдено в head: 28876 (72.2%)
Найдено в content: 1412 (дополнительно)
Отфильтровано гео-названий: 0
Итоговых замен id_hash: 28876 (72.2%)

Примеры успешных замен:
                                                                         head                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [ ]:
import pandas as pd
 # Очистка текста
    text = re.sub(r'^\s*Re:\s*', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\(.*?\)', '', text)
    text = re.sub(r'[«»"“”]', '', text)
    
    # Удаление статусных слов
    text = re.sub(
        r'\b(пропал[а-яё]*|жив[а-яё]*|погиб[а-яё]*|пропаж[а-яё]*|стоп\b)', 
        '', 
        text, 
        flags=re.IGNORECASE
    )
def print_empty_id_hash_rows(file_path):
    try:
        # Загрузка файла с автоматическим определением кодировки
        try:
            df = pd.read_csv(file_path, encoding='utf-8')
        except UnicodeDecodeError:
            df = pd.read_csv(file_path, encoding='cp1251')
        
        # Проверка наличия колонки id_hash
        if 'id_hash' not in df.columns:
            print("Ошибка: колонка 'id_hash' не найдена в файле")
            return
        
        # Находим строки с пустым id_hash
        empty_mask = (df['id_hash'].isna()) | (df['id_hash'].astype(str).str.strip() == '')
        empty_rows = df[empty_mask]
        
        # Выводим результат
        print(f"Найдено строк с пустым id_hash: {len(empty_rows)}")
        
        if not empty_rows.empty:
            print("\nСтроки с пустым id_hash:")
            # Выводим все колонки для наглядности
            pd.set_option('display.max_columns', None)
            pd.set_option('display.expand_frame_repr', False)
            print(empty_rows.to_string(index=False))
        else:
            print("Строк с пустым id_hash не найдено")
            
    except FileNotFoundError:
        print(f"Ошибка: файл {file_path} не найден")
    except Exception as e:
        print(f"Произошла ошибка: {str(e)}")

# Использование
if __name__ == "__main__":
    file_path = "updated_file.csv"  # Укажите путь к вашему файлу
    print_empty_id_hash_rows(file_path)
   

Найдено строк с пустым id_hash: 441

Строки с пустым id_hash:
 Unnamed: 0 id_hash gender  age                location    status        date_search     date_of_loss last_search_date search_period                                                                                          head                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        